In [ ]:
!pip install p_tqdm
!pip install dcor
!pip install iisignature
!pip install quantstats

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import LinearRegression
from preprocessing_data.preprocessing import  selected_uni ,preprocess_ffill,resample_crypto_data
from models.LeadLag_main import LeadLagAnalyzer
root_path = '../LeadLag_signature'

In [ ]:
df_universe = pd.read_csv(f'{root_path}/raw_data/universe_data.csv')
df_MTO = pd.read_csv(f'{root_path}/raw_data/Market_Turnover.csv')

close_price = pd.read_csv(f'{root_path}/raw_data/1H_prices_20250811.csv')
close_price = close_price.set_index('date')

df_MTO = df_MTO.set_index('date')


close_price = close_price.rename_axis("date")
df_MTO = df_MTO.rename_axis("date")

close_price.index = pd.to_datetime(close_price.index)
df_MTO.index = pd.to_datetime(df_MTO.index)

df_MTO = df_MTO.drop(columns=["datetime"])
if 'datetime' in close_price.columns:
  close_price = close_price.drop(columns=["datetime"])

df_MTO = df_MTO.ffill()
df_universe.iloc[0]['symbols']

In [ ]:
close_price = resample_crypto_data(close_price, '1D', 'close')
# close_price = resample_crypto_data(close_price, '4H', 'close')

# close_price1 = resample_crypto_data(close_price, '4H', 'close')
close_price

In [ ]:
universe = selected_uni(close_price , df_universe , maximum_coin=20 , window_size=210)
universe

In [ ]:
close_price , universe = preprocess_ffill(close_price,universe , end_date='2025-01-01')
close_price

LeadLag Detection


In [ ]:
from pickle import TRUE
LeadLag_config = {
    'method': 'dtw',  # 'ccf_at_lag', 'ccf_auc', 'signature', 'ccf_at_max_lag'
    'lookback': 30,
    'update_freq': 1,
    'use_parallel': False,
    'num_cpus': 7,
    'Scaling_Method': 'mean-centering', #mean-centering or by_std

    # Method-specific configurations
    'ccf_at_lag': {
        'lag': 3,
        'correlation_method': 'squared_pearson',    # 'pearson', 'kendall', 'spearman', 'distance', 'mutual_information', 'squared_pearson'
        'quantiles': 7                      # Only needed if using mutual_information correlation
    },

    'ccf_auc': {
        'max_lag': 15,                      # Required for ccf_auc
        'correlation_method': 'mutual_information',
        'quantiles': 5
    },

    'ccf_at_max_lag': {
        'max_lag': 5,
        'correlation_method': 'kendall',
        'quantiles': 3
    },

    'signature': {
        'sig_method' : 'custom', # levy or custom
        'correlation_method': 'mutual_information',    # for consistency

    },
}

In [ ]:
analyzer = LeadLagAnalyzer(LeadLag_config,universe)
single_matrix = analyzer.analyze(close_price, return_rolling=True)


In [ ]:
single_matrix.iloc[500]

In [ ]:
leader_follower_detector_config={
    'method': 'percentile', # percentile
    'agg_func': 'mean' , #sum or mean

    # percentile parameter
    'top_percentile': 50.0,
    'bottom_percentile': 50.0,
}

In [ ]:
leader_followers = analyzer.leader_follower_detector(single_matrix , leader_follower_detector_config)

In [ ]:

leader_followers.iloc[0]